In [1]:
!pip install keras-tuner

In [2]:
# Import our dependencies
import pandas as pd
import matplotlib as plt
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

## 1. Import & Inspect the Data

In [3]:
# # Connect to Google Drive
# from google.colab import drive 
# drive.mount('/content/gdrive')

In [4]:
# # Loading data
# tornado_df = pd.read_csv("gdrive/My Drive/Colab Notebooks/Final_Project/2008-2020_tornadoes_EF_cleaned.csv")
# tornado_df

In [5]:
# Apache Spark Set Up

import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,498 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,272 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd

In [6]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tornadoes").getOrCreate()

In [7]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://group-2-project-tornadoes.s3.amazonaws.com/2008-2020_tornadoes_EF_cleaned_db.csv"
spark.sparkContext.addFile(url)
tornado = spark.read.csv(SparkFiles.get("2008-2020_tornadoes_EF_cleaned_db.csv"), sep=",", header=True)

# Show DataFrame
tornado.show()

+-------+---------+----+-----+---+----------------+-------+---------+---------+---------+-------+-------+---+--------+----------+-------+---------+------+-----+
|Torn_ID|County_ID|Year|Month|Day|       Timestamp|ST_FIPS|CNTY_FIPS|Start_Lat|Start_Lon|End_Lat|End_Lon| EF|Injuries|Fatalities|   Loss|Crop_Loss|Length|Width|
+-------+---------+----+-----+---+----------------+-------+---------+---------+---------+-------+-------+---+--------+----------+-------+---------+------+-----+
|      1|    35049|2008|   10| 11|10/11/2008 18:33|     35|       49|    35.15|  -105.94|      0|      0|  0|       0|         0|  12000|     null|  0.02|   10|
|      2|    20181|2008|   10| 11|10/11/2008 21:54|     20|      181|    39.33|  -101.55|      0|      0|  0|       0|         0|   null|     null|  0.25|   10|
|      3|    13039|2008|   10| 24|10/24/2008 16:55|     13|       39|    31.02|   -81.81|      0|      0|  0|       0|         0|   1000|     null|  0.25|   25|
|      4|     5113|2008|   10|  6|

In [8]:
# Convert spark object to pandas dataframe
tornado_df = tornado.toPandas()
tornado_df

,Torn_ID,County_ID,Year,Month,Day,Timestamp,ST_FIPS,CNTY_FIPS,Start_Lat,Start_Lon,End_Lat,End_Lon,EF,Injuries,Fatalities,Loss,Crop_Loss,Length,Width
0,1,35049,2008,10,11,10/11/2008 18:33,35,49,35.15,-105.94,0,0,0,0,0,12000,None,0.02,10
1,2,20181,2008,10,11,10/11/2008 21:54,20,181,39.33,-101.55,0,0,0,0,0,None,None,0.25,10
2,3,13039,2008,10,24,10/24/2008 16:55,13,39,31.02,-81.81,0,0,0,0,0,1000,None,0.25,25
3,4,5113,2008,10,6,10/6/2008 13:46,5,113,34.58,-94.15,34.63,-94.12,1,0,0,75000,None,3.41,250
4,5,48067,2008,10,6,10/6/2008 16:05,48,67,33.07,-94.1,33.09,-94.08,0,0,0,75000,None,1.5,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15886,15887,47095,2020,9,1,9/1/2020 16:24,47,95,36.329,-89.4981,36.3517,-89.4729,0,0,0,None,2000,2.1,50
15887,15888,24003,2020,9,3,9/3/2020 15:57,24,3,38.948,-76.5695,38.9233,-76.4615,0,0,0,1250000,None,6.05,100
15888,15889,6039,2020,9,5,9/5/2020 16:17,6,39,37.35,-119.34,37.5218,-119.3056,2,0,0,None,None,12.02,50
15889,15890,6019,2020,9,5,9/5/2020 18:29,6,19,37.25,-119.2,37.2598,-119.1838,1,0,0,None,None,1.12,25


In [9]:
# Check data types
tornado_df.dtypes

Torn_ID       object
County_ID     object
Year          object
Month         object
Day           object
Timestamp     object
ST_FIPS       object
CNTY_FIPS     object
Start_Lat     object
Start_Lon     object
End_Lat       object
End_Lon       object
EF            object
Injuries      object
Fatalities    object
Loss          object
Crop_Loss     object
Length        object
Width         object
dtype: object

In [10]:
# Convert timestamp to datetime
tornado_df['Timestamp']= pd.to_datetime(tornado_df['Timestamp'])

# Convert all strings to numeric
cols=[i for i in tornado_df.columns if i not in ["Timestamp"]]
for col in cols:
    tornado_df[col]=pd.to_numeric(tornado_df[col])

# check dtypes
tornado_df.dtypes

Torn_ID                int64
County_ID              int64
Year                   int64
Month                  int64
Day                    int64
Timestamp     datetime64[ns]
ST_FIPS                int64
CNTY_FIPS              int64
Start_Lat            float64
Start_Lon            float64
End_Lat              float64
End_Lon              float64
EF                     int64
Injuries               int64
Fatalities             int64
Loss                 float64
Crop_Loss            float64
Length               float64
Width                  int64
dtype: object

In [11]:
# drop na columns
tornado_df = tornado_df.dropna()

### 1. Define input data (X) and output data (y)

In [12]:
X = tornado_df.copy()
X = X.drop(['Torn_ID','County_ID','Year', 'Month', 'Day', 'Timestamp','ST_FIPS','CNTY_FIPS','Start_Lat','Start_Lon','End_Lat', 'End_Lon', 'Fatalities', 'Loss',	'Crop_Loss'], axis=1)
X

,EF,Injuries,Length,Width
22,1,0,8.13,200
28,2,2,4.09,150
29,3,4,8.19,100
73,1,0,10.59,500
81,1,0,3.14,250
...,...,...,...,...
15764,0,0,1.26,50
15785,1,1,0.26,50
15786,1,0,2.17,100
15838,0,0,2.58,100


In [13]:
y = tornado_df["Fatalities"].ravel()
y[:10]

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

### 2. Splitting into Train and Test sets

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  random_state =78)

### 3. Scale the input data (X)

In [15]:
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### 4. Define Optimizer

In [16]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=len(X.columns) ) )

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="mean_squared_error", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [17]:
# Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [18]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

INFO:tensorflow:Oracle triggered exit


In [19]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 3,
 'num_layers': 5,
 'tuner/bracket': 2,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units_0': 5,
 'units_1': 7,
 'units_2': 5,
 'units_3': 7,
 'units_4': 7}

In [20]:
best_model = tuner.get_best_models()[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 15        
                                                                 
 dense_1 (Dense)             (None, 5)                 20        
                                                                 
 dense_2 (Dense)             (None, 7)                 42        
                                                                 
 dense_3 (Dense)             (None, 5)                 40        
                                                                 
 dense_4 (Dense)             (None, 7)                 42        
                                                                 
 dense_5 (Dense)             (None, 7)                 56        
                                                                 
 dense_6 (Dense)             (None, 1)                 8

In [21]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5/5 - 0s - loss: 0.8076 - accuracy: 0.9688 - 393ms/epoch - 79ms/step
Loss: 0.807648777961731, Accuracy: 0.96875
